In [ ]:
!pip install tensorflow-text
!pip install datasets

     |████████████████████████████████| 4.3MB 8.2MB/s 
     |████████████████████████████████| 235kB 8.8MB/s 
     |████████████████████████████████| 245kB 25.4MB/s 
     |████████████████████████████████| 112kB 37.3MB/s 


In [ ]:
from datasets import load_dataset
from gensim.utils import tokenize

my_dataset = load_dataset("amazon_reviews_multi", "es", split='train')
my_dataset = my_dataset.filter(lambda example: example['stars'] in [1, 5])
my_dataset = [list(tokenize(example['review_body'])) for example in my_dataset]


Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


In [ ]:
from gensim.models import word2vec
w2v = word2vec.Word2Vec(my_dataset, size=100, min_count=10)

In [ ]:
print(w2v.wv['maravilloso'])

[-0.03379149 -0.18464535  0.2242653   0.31616217  0.38063487 -0.06143043
  0.12892932  0.2649963  -0.12212728  0.03489837  0.18577786 -0.05714865
  0.11969813 -0.47349843 -0.0630132   0.24415861 -0.28433323 -0.10434402
 -0.09021619  0.47066376  0.15089284 -0.5743117   0.00372268 -0.14313978
 -0.01951659  0.2017641   0.25342515  0.42408484  0.2072472  -0.13011217
  0.07902407  0.30504102  0.30578837  0.0809433   0.08886173 -0.12824473
 -0.55605525  0.01107993 -0.17158332  0.33708617 -0.3148922   0.02689911
  0.11952768  0.03760486  0.07621975  0.07869417 -0.34089628 -0.6116333
 -0.05992611  0.09615379  0.01514725 -0.04867911 -0.01035369 -0.10439476
  0.05309996  0.28642663  0.16890453  0.16225609 -0.3553835   0.03066736
 -0.29274496 -0.07034983 -0.0707289  -0.09177589 -0.02304501  0.00625305
  0.225435   -0.14645562 -0.2617115   0.11860451 -0.37121814 -0.25750208
  0.00435543 -0.12386144 -0.01903593  0.22466286 -0.08811922  0.04300737
  0.06896928  0.02915672  0.20444955 -0.18045518 -0.

In [ ]:
print(w2v.wv['maravill0so'])

KeyError: ignored

In [ ]:
from gensim.models import fasttext
ft = fasttext.FastText(size=100, window=3, min_count=10)
ft.build_vocab(sentences=my_dataset)
ft.train(sentences=my_dataset, total_examples=len(my_dataset), epochs=10)

In [ ]:
print(ft.wv['maravill0so'])

[ 9.2096692e-01 -1.5862778e+00  1.0146941e+00  4.8887351e-01
 -8.6543047e-01  2.5365356e-01  8.3066434e-01 -1.8039241e-01
 -1.2630411e+00  1.6499874e-01 -6.0046023e-01 -1.6828839e+00
  1.1958090e+00 -3.8922796e-01  2.3995806e-01 -6.3484472e-01
 -1.9568348e-01  8.4391028e-01  1.1496909e+00  1.8799119e-01
  3.0394837e-01 -2.4619886e-01 -1.0054247e+00  5.1235574e-01
  3.3774322e-01 -1.3979940e+00 -2.0591114e-01 -1.6496366e+00
  2.7822253e-01  4.3580386e-01 -5.9395198e-02 -1.1302497e+00
 -5.7009447e-01  9.4624138e-01  2.9959962e-01  1.6515131e-01
 -8.2142508e-01  4.8149151e-01 -6.0059905e-02  7.1361887e-01
 -9.9153632e-01  6.5248221e-01  3.9460264e-02 -1.3819708e+00
  1.7251247e+00 -1.5779973e+00 -6.7241853e-01  6.1384714e-01
  5.4913157e-01  4.2635697e-01  7.6677114e-02  9.9999011e-01
 -3.8374993e-01  5.5813682e-01  9.0808250e-02  1.0513518e+00
 -6.8074095e-01 -6.7762381e-01  3.6322412e-01 -5.8238733e-01
 -3.1595743e-01  4.4373980e-01 -3.9693177e-01 -4.1993573e-01
 -1.3256902e+00  6.14641

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print(cosine_similarity([ft.wv['marav1loso']], [ft.wv['maravilloso']]))

[[0.93692493]]


In [ ]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4", trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

embedding_model = tf.keras.Model(text_input, sequence_output)


In [ ]:
sentences = tf.constant(["But I like the product", "She sings like a bird", "I like the bird"])
embeddings = embedding_model(sentences)
print(embeddings.shape)

(3, 128, 768)


In [ ]:
print(cosine_similarity([embeddings[0][2]], [embeddings[1][2]]))
print(cosine_similarity([embeddings[0][2]], [embeddings[2][1]]))

[[0.43935663]]
[[0.8367212]]
